In [1]:
import pandas as pd

In [6]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [ ]:
users = pd.read_csv("./ml-100k/u.user", sep="|", names= u_cols)
rating_base = pd.read_csv("./ml-100k/ua.base",sep="\t",names = r_cols)
rating_test = pd.read_csv("./ml-100k/ua.test",sep="\t",names = r_cols)
items = pd.read_csv("./ml-100k/u.item", sep="|",names=i_cols,encoding="latin-1")
rate_base = rating_base.to_numpy()
rate_test = rating_test.to_numpy()
X0 = items.to_numpy()
n_items = items.shape[0]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 76620: invalid continuation byte

In [ ]:
print(rating_base.shape[0])
print(rating_base.head())
print(rate_base)
print(rate_test)

print(rate_base.shape[0])
print(rate_test.shape[0])
print()

90570
   user_id  movie_id  rating  unix_timestamp
0        1         1       5       874965758
1        1         2       3       876893171
2        1         3       4       878542960
3        1         4       3       876893119
4        1         5       3       889751712
[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 ...
 [      943      1188         3 888640250]
 [      943      1228         3 888640275]
 [      943      1330         3 888692465]]
[[        1        20         4 887431883]
 [        1        33         4 878542699]
 [        1        61         4 878542420]
 ...
 [      943       570         1 888640125]
 [      943       808         4 888639868]
 [      943      1067         2 875501756]]
90570
9430
